Quick Draw는 많은 사용자들이 그린 그림을 맞추는 게임이다.
class는 340개이지만 사람의 습관에 따라 같은 class도 달라진다는 것을 생각했을때 수많은 경우의 수를 생각하여 결과를 내어야 한다.

본 과제에서는 좌표형태의 데이터를 다시 이미지로 변환시킨다.
이미지 분류에 좋은 성능을 가지고 있는 CNN을 이용하여 데이터 학습을 진행하였다.

**Trainning model**


model의 input을 주기 위해 csv의 데이터에서 x,y점의 좌표를 읽어 전처리 작업을 행한다. 

In [ ]:
import os
import re
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fnames = glob('../input/train_simplified/*.csv') #<class 'list'>
cnames = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']
drawlist = []
for f in fnames[0:6]: # num of word : 5
    first = pd.read_csv(f, nrows=10) # make sure we get a recognized drawing
    first = first[first.recognized==True].head(2) # top head 2 get 
    drawlist.append(first)
draw_df = pd.DataFrame(np.concatenate(drawlist), columns=cnames) # <class 'pandas.core.frame.DataFrame'>

In [ ]:
draw_df.drawing.values[0]

In [ ]:
evens = range(0,11,2)
odds = range(1,12, 2)
# We have drawing images, 2 per label, consecutively
df1 = draw_df[draw_df.index.isin(evens)]
df2 = draw_df[draw_df.index.isin(odds)]

example1s = [ast.literal_eval(pts) for pts in df1.drawing.values]
example2s = [ast.literal_eval(pts) for pts in df2.drawing.values]
labels = df2.word.tolist()

for i, example in enumerate(example1s):
    plt.figure(figsize=(6,3))
    
    for x,y in example:
        plt.subplot(1,2,1)
        plt.plot(x, y, marker='.')
        plt.axis('off')

    for x,y, in example2s[i]:
        plt.subplot(1,2,2)
        plt.plot(x, y, marker='.')
        plt.axis('off')
        label = labels[i]
        plt.title(label, fontsize=10)

    plt.show()  

Keras
tensorflow 위에 구현된 keras를 사용하였다.
tensorflow 기반을 필두로 하여 keras를 사용하는데 큰 부담이 들지 않았다.


좌표를 이용해서 이미지를 변환하는 과정은 시간의 문제점이 있었다.
해서 데이터 셋의 배열의 크기를 바꾸고 연속된 좌표의 패턴을 학습하였다.

In [ ]:
import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
from PIL import Image, ImageDraw 
from tqdm import tqdm
from dask import bag
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.metrics import top_k_categorical_accuracy
from keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


In [ ]:
path = '../input/train_simplified/'
classfiles = os.listdir(path)

numstonames = {i: v[:-4].replace(" ", "_") for i, v in enumerate(classfiles)} # sleeping bag -> sleeping_bag
files = [os.path.join(path, file) for i, file in enumerate(classfiles)]
word_mapping = {file.split('/')[-1][:-4]:i for i, file in enumerate(files)}

num_classes = len(files)    #340
imheight, imwidth = 32, 32 # size of an image
ims_per_class = 2000  #max? # in the code above and above, there existed more than 100 thousand images per class(/label)
sequence_length = 80


**making train data**

총 340개의 class 개수가 있다.
한 class 마다 sleeping bag.csv 에서 15000 읽어오는 작업을 수해한다. (단, colum 'drawing', 'recognized'만 추출 온다)

이중 'recognized' 가 'True' 인 애들 탑 10000 개를 뽑는다.

X

sequence of x- 와 y-coordinates 의 패턴을 X 배열에 넣는다.
X 배열의 차원을 줄여 [10000, 160] 배열을 만든다.

y

index 값 중 class를 구별 할 수 있는 것만 넣어 [10000, 1] 배열을 만든다.

X와 y 배열을 결합한다. [10000, 161]

train_grand

x와 y를 결합한 것을 'train_grand'에 넣어주고 shape을 변환한다.
[340, 10000, 161] -> [3400000, 161]

In [ ]:
train_grand= []

class_paths = glob('../input/train_simplified/*.csv')

df = []

for i,c in enumerate(tqdm(class_paths[0: num_classes])):
    train = pd.read_csv(c, usecols=['drawing', 'recognized'], nrows=15000) # [2500 rows x 2 columns]
    train = train[train.recognized == True].head(10000) # use data only recognized == True -> [2000 rows x 2 columns]
    
    X = []
    for values in train.drawing.values:
        image = json.loads(values)
        strokes = []
        for x_axis, y_axis in image:
            strokes.extend(list(zip(x_axis, y_axis)))
        strokes = np.array(strokes)
        pad = np.zeros((sequence_length, 2))
        if sequence_length>strokes.shape[0]:
            pad[:strokes.shape[0],:] = strokes
        else:
            pad = strokes[:sequence_length, :]
        X.append(pad)
    X = np.array(X)
    y = np.full((train.shape[0], 1), i)
    X = np.reshape(X, (10000, -1))
    X = np.concatenate((y, X), axis=1)
    train_grand.append(X)
   
    
train_grand = np.array([train_grand.pop() for i in np.arange(num_classes)]) 
print(train_grand.shape)
train_grand = train_grand.reshape((-1, sequence_length*2+1))
print(train_grand.shape)

del X
del train

Define model using keras 

* 입력 : 채널 한개의 (32,32)
* 다리 : 필터개수 32개의 (3,3)
* 출력 : 채널 32의 (32,32)

지역적 특징 추출을 위해 1D filters 이용

In [ ]:
def createNetwork(seq_len):
    
    # Function to add a convolution layer with batch normalization
    def addConv(network, features, kernel):
        network = BatchNormalization()(network)
        return Conv1D(features, kernel, padding='same', activation='relu')(network)
    
    # Function to add a dense layer with batch normalization and dropout
    def addDense(network, size):
        network = BatchNormalization()(network)
        network = Dropout(0.2)(network)
        return Dense(size, activation='relu')(network)
    
    
    # Input layer
    input = Input(shape=(seq_len, 2))
    network = input
    
    # Add 1D Convolution
    for features in [16, 24, 32]:
        network = addConv(network, features, 5)
    network = MaxPool1D(pool_size=5)(network)
    
    # Add 1D Convolution
    for features in [64, 96, 128]:
        network = addConv(network, features, 5)
    network = MaxPool1D(pool_size=5)(network)

    # Add 1D Convolution
    for features in [256, 384, 512]:
        network = addConv(network, features, 5)
    #network = MaxPool1D(pool_size=5)(network)

    # Flatten
    network = Flatten()(network)
    
    # Dense layer for combination
    for size in [128, 128]:
        network = addDense(network, size)
    
    # Output layer
    output = Dense(len(files), activation='softmax')(network)


    # Create and compile model
    model = Model(inputs = input, outputs = output)



    return model

model = createNetwork(sequence_length)

**모델 사용**

model.compile()


다중 클래스 -> ‘categorical_crossentropy’
경사 하강법 알고리즘 ‘adam’을 사용
평가 척도를 의미하며 분류 문제는 ‘accuracy'(일반적)


model.fit()

모델 학습

훈련 데이터셋 , batch 사이즈, epoch 수, 검증 데이터셋, 학습 중 출력되는 문구등을 내포

In [ ]:
def top_3_accuracy(x,y): 
    t3 = top_k_categorical_accuracy(x,y, 3)
    return t3

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='auto', min_delta=0.005, cooldown=5, min_lr=0)

earlystop = EarlyStopping(monitor='val_loss', mode='auto', patience=2,verbose=0) 


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', top_3_accuracy])

model.summary()

model.fit(x=X_train, y=y_train,
          batch_size = 1000,
          epochs = 25,
          validation_data = (X_val, y_val),
          verbose = 1)

In [ ]:
ttvlist = []
reader = pd.read_csv('../input/test_simplified.csv', index_col=['key_id'],
    chunksize=2048)

for chunk in tqdm(reader, total=55):
    X =[]
    for values in chunk.drawing.values:
        image = json.loads(values)
        strokes = []
        for x_axis, y_axis in image:
            strokes.extend(list(zip(x_axis, y_axis)))
        strokes = np.array(strokes)
        pad = np.zeros((sequence_length, 2))
        if sequence_length>strokes.shape[0]:
            pad[:strokes.shape[0],:] = strokes
        else:
            pad = strokes[:sequence_length, :]
        X.append(pad)
        
    X = np.array(X)
    X = np.reshape(X, (-1,sequence_length, 2))
    testpreds = model.predict(X, verbose=0)
    ttvs = np.argsort(-testpreds)[:, 0:3]
    ttvlist.append(ttvs)

    
ttvarray = np.concatenate(ttvlist)


In [ ]:
preds_df = pd.DataFrame({'first': ttvarray[:,0], 'second': ttvarray[:,1], 'third': ttvarray[:,2]})
preds_df = preds_df.replace(numstonames)
preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

sub = pd.read_csv('../input/sample_submission.csv', index_col=['key_id'])
sub['word'] = preds_df.words.values
sub.to_csv('submission_cnn.csv')
sub.head()